In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import matplotlib.pyplot as plt                 
import matplotlib.image as mpimg
training_data_dir= '/kaggle/input/leaf-classification/dataset/train'     #importing training data
validation_data_dir='/kaggle/input/leaf-classification/dataset/test'     #importing test data 
IMAGE_SIZE = 224                                                      
IMAGE_WIDTH, IMAGE_HEIGHT = IMAGE_SIZE, IMAGE_SIZE                       # Size of image = 224*224*3
EPOCHS = 25                                                              # Number of epoches 
BATCH_SIZE = 64                                                          # Size of batch 
TEST_SIZE = 30                                            


In [ ]:
input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 3)   # Size of Image = [224,224,3]
import tensorflow as tf                        # Importing tensorflow
import tensorflow.keras                        # Importing Keras from tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator        #importing Image-data-generator class from kreas that is used for augmentation of image
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, SeparableConv2D, UpSampling2D # importing attributes that will be used for making layers 
from tensorflow.keras.models import Model
                                                                                                                             
from tensorflow.keras.layers import  GlobalAveragePooling2D , Input ,  BatchNormalization   #import attributes that will be used for making layers  



training_data_generator = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=60 ,
    brightness_range=[0.5 , 1.0]
    
    )                                                 #training data image augmentation 
validation_data_generator = ImageDataGenerator(rescale=1./255,
         horizontal_flip=True,
    vertical_flip=True,
    rotation_range=60,
    brightness_range=[0.5 , 1.0])                    #validation data augmentation 
training_generator = training_data_generator.flow_from_directory(
    training_data_dir,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle = True,
    class_mode="categorical")                      #generating tarining data

validation_generator = validation_data_generator.flow_from_directory(
    validation_data_dir,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle = True,
    class_mode="categorical")                   #generating validation data 
 

In [ ]:
# In Xception model ,The data first goes through the entry flow, then through the middle flow , and finally through the exit flow ,so here is the entery flow.
def entry_flow(inputs) :                   # X

    x = Conv2D(32,(3,3), strides = 2, padding='valid')(inputs)       # 2D convlational layer with size (32,3*3) , strides =2*2
    x = BatchNormalization()(x)                                     # Applying batch norm on (X) to imporve network training 
    x = Activation('relu')(x)                                       # relu Activation on batch-normalised layer 


    x = Conv2D(64,(3,3),padding='same')(x)                          #2D convulational layer with size (32, 3*3) , strides = 1*1
    x = BatchNormalization()(x)                                      # Applying batch norm on (X) to imporve network training    
    x = Activation('relu')(x)                                        # relu Activation on batch-normalsied layer 
    x=Dropout(0.2)(x)                                                #
    previous_block_activation = x                                    #saving activation of this layer to a variable known as privious_block_activation 

    for size in [128, 256, 728] :                                    

        x = Activation('relu')(x)                                     # relu Activation
        x = SeparableConv2D(size, 3, padding='same')(x)               # Separable Convolution 2D layer of Size=size
        x = BatchNormalization()(x)                                    # Applying batch norm on (X) to imporve network training 

        x = Activation('relu')(x)                                    # relu Activation on x
        x = SeparableConv2D(size, 3, padding='same')(x)              # Separable Convolution 2D layer of Size=size
        x = BatchNormalization()(x)                                  # Applying batch norm on (x) to imporve network training 

        x = MaxPooling2D(3, strides=2, padding='same')(x)           # applying maxpooling to x with size 3*3
        x=Dropout(0.2)(x)                                           # dropout on x with value 0.2
        residual = Conv2D(size, 1, strides=2, padding='same')(previous_block_activation)        # forming a convulational layer on previous activation block and storing in residual 

        x = tensorflow.keras.layers.Add()([x, residual])           # combining x and residual 
        previous_block_activation = x                           

    return x
# next is the middle layer of the xception model 
def middle_flow(x, num_blocks=8) : # x is the input taht came from output of the entry layer , num_block=8 represents that in  the  middle_flow the training is done 8 times  
                              
    previous_block_activation = x
    ##teh process of depthwise convulation is repeated 8 times as it was done for entry_flow function .
    for _ in range(num_blocks) :

        x = Activation('relu')(x)                                     
        x = SeparableConv2D(728, (3,3), padding='same')(x)         
        x = BatchNormalization()(x)

        x = Activation('relu')(x)
        x = SeparableConv2D(728,( 3,3), padding='same')(x)
        x = BatchNormalization()(x)
       
        x = Activation('relu')(x)
        x = SeparableConv2D(728, (3,3), padding='same')(x)
        x = BatchNormalization()(x)
        x=Dropout(0.2)(x)
        x = tensorflow.keras.layers.Add()([x, previous_block_activation])
        previous_block_activation = x

    return x

# after the middle_flow comes the exit_flow
# in the exit folw the depthwise convulation is applied as it was applied in entry_flow and middle_flow

def exit_flow(x) :
   
    previous_block_activation = x

    x = Activation('relu')(x)
    x = SeparableConv2D(728,( 3,3), padding='same')(x)
    x = BatchNormalization()(x)
    
    x = Activation('relu')(x)
    x = SeparableConv2D(1024, (3,3), padding='same')(x) 
    x = BatchNormalization()(x)

    x = MaxPooling2D(3, strides=2, padding='same')(x)
    x=Dropout(0.2)(x)
    residual = Conv2D(1024, (1,1), strides=2, padding='same')(previous_block_activation)
    x = tensorflow.keras.layers.Add()([x, residual])

    x = Activation('relu')(x)
    x = SeparableConv2D(728, (3,3), padding='same')(x)
    x = BatchNormalization()(x)

    x = Activation('relu')(x)
    x = SeparableConv2D(1024,(3,3), padding='same')(x)
    x = BatchNormalization()(x)
    x=Dropout(0.2)(x)
    x = GlobalAveragePooling2D()(x)         # here avergae pooling is done instead of max pooiling as it retains more results than avergae pooling 
    
    x = Dense(185, activation='softmax')(x)    # finally the output is being computed by softmax funtion

    return x


In [ ]:
inputs = Input(shape=(224, 224, 3))   
outputs = exit_flow(middle_flow(entry_flow(inputs)))  # calling output function 
xception = Model(inputs, outputs)             # creating an modle named xception out of input and output from output from the exit_flow
#model=xception.summary()
from tensorflow.keras.callbacks import EarlyStopping    # importing earlystopping from kears.callbacks 
#es=tensorflow.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,verbose=1)
xception.compile(optimizer = tensorflow.keras.optimizers.Adam(lr = 0.001), loss='categorical_crossentropy', metrics=['accuracy'])  # compiling the model using adam optimiser setting 
                                                                                                                                   # learning rate =0.01 , and setting loss as categorical crossentropy as more than 2 classes are being used to train 
                                                                                                                                   #setting metrices to accuracy to monitor acuracy 




In [ ]:
history=xception.fit_generator(
    training_generator,
    steps_per_epoch=len(training_generator.filenames) // BATCH_SIZE,
    epochs=15,
    validation_data=validation_generator,
    validation_steps=len(validation_generator.filenames) // BATCH_SIZE)

### training the model on training data for 15 epoches  with optimiser as Adam with learning rate =0.01




In [ ]:

xception.compile(optimizer = tensorflow.keras.optimizers.Adam(lr = 0.001),
              loss='categorical_crossentropy', metrics=['accuracy',])
history=xception.fit_generator(
    training_generator,
    steps_per_epoch=len(training_generator.filenames) // BATCH_SIZE,
    epochs=15,
    validation_data=validation_generator,
    validation_steps=len(validation_generator.filenames) // BATCH_SIZE
           )     
### training the model on training data for 15 epoches  with optimiser as Adam with learning rate =0.001



In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint     # importing ModleCheckpoint from keras.callbacks
checkpointer = ModelCheckpoint(filepath='model.h5',monitor = 'val_loss', verbose=1, save_best_only=True) #assigning a checkpointer to save weights to model.h5 file and mointor validation loss 
callbacks = [ checkpointer]                                                                              # save_best_only=True means save weights only if the validation loss improved in that epoch

xception.compile(optimizer = tensorflow.keras.optimizers.Adam(lr = 0.0001),
              loss='categorical_crossentropy', metrics=['accuracy',])   #  compiling the model with adam optimiser with learning rate =0.0001 . 
history=xception.fit_generator(
    training_generator,
    steps_per_epoch=len(training_generator.filenames) // BATCH_SIZE,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=len(validation_generator.filenames) // BATCH_SIZE,
   callbacks = [ checkpointer]        )
# training the model with training data and validating using validation data and also applying a checkpointer .